# Using Great Circle distance to find closest competitors

This notebook helps you calculate great circle distances to identify close or the closest other locations to a set of 'source locations'. This uses Latitude and Longitude information -- if you only have street addresses, you need to geocode them, see the HERE Maps API for examples. 

The distance calculations functions are defined within the notebook. In future versions, we will move them to the utilities folder into re-usable .py files.

For questions, email daniel.wang@oliverwyman.com

**Import Python libaries**

In [1]:
import pandas as pd
import numpy as np

# Load in and prep the data

In this example, we have 1 input table with locations for both the 'sources' and 'targets'

In [2]:
dataset = pd.read_csv("sample_input/inputs.csv", low_memory=False)

In [3]:
from_data = dataset.loc[dataset['Category'] == "A"].reset_index(drop=True)
from_data.shape

(6249, 4)

In [4]:
to_data = dataset.loc[dataset['Category'] != "B"].reset_index(drop=True)
to_data.shape

(16769, 4)

# Define functions for great circle distance calculation

In [5]:
# Generic function for calculating great circle distance, vectorized
def great_circle_distance(lon1, lat1, lon2, lat2, unit='miles'):
    """    
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """

    # convert decimal degrees to radians    
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])    # haversine formula    
    dlon = lon2 - lon1    
    dlat = lat2 - lat1    
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2    
    c = 2 * np.arcsin(np.sqrt(a))    
    
    if unit in ["miles", "mile"]:
        r = 3958.756 # Radius of earth in miles
    elif unit in ["km", "kilometer", "kilometers"]:
        r = 6371.0 # Radius of earth in kms

    return c * r

In [6]:
# Function for getting closest distance only.  
def get_closest(origin, targets, lon_column="Lon", lat_column="Lat", unit='miles', append_origin=False):
    """
    Great circle distance between an origin (single row in dataframe) and a dataframe with target destinations. 
    lon_column is the column name containing Longitude
    lat_column is the column name containing Latitude
    append_origin should be true to add the origin information
    """
    
    targets['great_circle_distance'] = great_circle_distance(origin[lon_column], origin[lat_column], 
                                                             targets[lon_column], targets[lat_column])
    
    # Find the row with the closest
    minimum = pd.DataFrame(targets.iloc[targets['great_circle_distance'].argmin()]).T
    
    if append_origin:
        for col in origin.index:
            minimum["Origin: {}".format(col)] = origin[col]

    return minimum


In [7]:
# Function for getting all within certain distance threshold
def get_within_threshold(origin, targets, threshold, always_include_closest=False, 
                         lon_column="Lon", lat_column="Lat", unit='miles', append_origin=False):
    """
    Great circle distance between an origin (single row in dataframe) and a dataframe with target destinations. 
    threshold is the max distance in units
    if always_include_closest is True, the closest is always included, even if outside the threshold
    lon_column is the column name containing Longitude
    lat_column is the column name containing Latitude
    """
    
    targets['great_circle_distance'] = great_circle_distance(origin[lon_column], origin[lat_column], 
                                                             targets[lon_column], targets[lat_column])
    
    # Get all within threshold
    within_threshold = targets.loc[targets['great_circle_distance'] <= threshold].sort_values("great_circle_distance").reset_index(drop=True)
    
    if within_threshold.shape[0]:
        minimum = within_threshold['great_circle_distance'].argmin()
        within_threshold['closest'] = False
        within_threshold.loc[minimum,'closest'] = True   # using loc is acceptable as the index has been reset
        output = within_threshold
        
    else:
        if always_include_closest:
            # Find the row with the closest
            minimum = targets['great_circle_distance'].argmin()
            output = pd.DataFrame(targets.iloc[minimum]).T
            output['closest'] = True
    
    if append_origin:
        for col in origin.index:
            output["Origin: {}".format(col)] = origin[col]
    
    return output

# Get closest location only

We can use apply() on a dataframe, or we can use a loop. The advantage with the loop is that it tracks progress, and if it breaks midway, we can continue where it left off by modifying the loop and keeping the output_list.

If you are working with very large tables, using apply or a parallel processing method is preferred.

In [8]:
output_list = []

nrows = from_data.shape[0]
for index, row in from_data.iterrows():
    
    if index % 500==0: print("Processing {} out of {} items".format(index+1, nrows+1))
    
    result = get_closest(row, targets = to_data, lon_column="Longitude", lat_column="Latitude", unit='miles', append_origin=True)
    
    output_list.append(result)

Processing 1 out of 6250 items
Processing 501 out of 6250 items
Processing 1001 out of 6250 items
Processing 1501 out of 6250 items
Processing 2001 out of 6250 items
Processing 2501 out of 6250 items
Processing 3001 out of 6250 items
Processing 3501 out of 6250 items
Processing 4001 out of 6250 items
Processing 4501 out of 6250 items
Processing 5001 out of 6250 items
Processing 5501 out of 6250 items
Processing 6001 out of 6250 items


**If the code above was interrupted, and you want to continue from where it left off, then run this code below (uncomment it first)**

In [9]:
# start_row = len(output_list)
# for index, row in from_data.reset_index(drop=True).iterrows():
#     if index < start_row:
#         continue
#     if index % 100==0: print("Processing {} out of {} items".format(index+1, nrows+1))
    
#     result = get_closest(row, targets = to_data, lon_column="Longitude", lat_column="Latitude", unit='miles', append_origin=True)
    
#     output_list.append(result)

In [10]:
closest_results = pd.concat(output_list)
closest_results.head()

,ID,Category,Latitude,Longitude,great_circle_distance,Origin: ID,Origin: Category,Origin: Latitude,Origin: Longitude
4,5,A,40.838419,-73.811787,0.0,5,A,40.838419,-73.811787
5,6,A,48.921904,-122.312536,0.0,6,A,48.921904,-122.312536
8,9,A,40.058377,-75.13899,0.0,9,A,40.058377,-75.138990
9,10,A,33.978845,-117.678706,0.0,10,A,33.978845,-117.678706
13,15,A,43.494928,-73.398742,0.0,15,A,43.494928,-73.398742


**Save results into csv**. Rename the output as needed.

In [11]:
closest_results.to_csv("sample_output/closest_competitor.csv")

# Get all distances within a certain threshold

We can use apply() on a dataframe, or we can use a loop. The advantage with the loop is that it tracks progress, and if it breaks midway, we can continue where it left off by modifying the loop and keeping the output_list. 

If you are working with very large tables, using apply or a parallel processing method is preferred.

**The example below uses a threshold of 10 miles, but you can modify this as needed**

In [12]:
output_list = []
nrows = from_data.shape[0]
for index, row in from_data.iterrows():
    
    if index % 500==0: print("Processing {} out of {} items".format(index+1, nrows+1))
    
    result = get_within_threshold(row, targets = to_data, 
                                  threshold = 10, always_include_closest=True,
                                  lon_column="Longitude", lat_column="Latitude", unit='miles', append_origin=True)
    
    output_list.append(result)

Processing 1 out of 6250 items
Processing 501 out of 6250 items
Processing 1001 out of 6250 items
Processing 1501 out of 6250 items
Processing 2001 out of 6250 items
Processing 2501 out of 6250 items
Processing 3001 out of 6250 items
Processing 3501 out of 6250 items
Processing 4001 out of 6250 items
Processing 4501 out of 6250 items
Processing 5001 out of 6250 items
Processing 5501 out of 6250 items
Processing 6001 out of 6250 items


**If the code above was interrupted, and you want to continue from where it left off, then run this code below (uncomment it first)**

In [13]:
# start_row = len(output_list)
# for index, row in from_data.reset_index(drop=True).iterrows():
#     if index < start_row:
#         continue
#     if index % 100==0: print("Processing {} out of {} items".format(index+1, nrows+1))
    
#     result = get_within_threshold(row, targets = to_data, 
#                                   threshold = 10, always_include_closest=True,
#                                   lon_column="Longitude", lat_column="Latitude", unit='miles', append_origin=True)
    
#     output_list.append(result)

In [14]:
results_within_threshold = pd.concat(output_list)
results_within_threshold.head()

,ID,Category,Latitude,Longitude,great_circle_distance,closest,Origin: ID,Origin: Category,Origin: Latitude,Origin: Longitude
0,5,A,40.838419,-73.811787,0.000000,True,5,A,40.838419,-73.811787
1,124,A,40.843095,-73.805346,0.466632,False,5,A,40.838419,-73.811787
2,20657,A,40.826284,-73.808752,0.853312,False,5,A,40.838419,-73.811787
3,10779,A,40.840593,-73.795423,0.868447,False,5,A,40.838419,-73.811787
4,13352,C,40.822925,-73.797612,1.301966,False,5,A,40.838419,-73.811787


**Save results into csv**. Rename the output as needed.

In [15]:
results_within_threshold.to_csv("sample_output/results_within_10miles_or_closest.csv")